In [30]:
from __future__ import unicode_literals, print_function

import random
from pathlib import Path
import spacy
from tqdm import tqdm
from spacy.training.example import Example
import pickle

In [31]:
model = None
output_dir=Path("ner/")
n_iter=100

#load the model

if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

Created blank 'en' model


In [32]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')


In [33]:
import json
# Opening JSON file
f = open('training_data.json')
  
# returns JSON object as 
# a dictionary

x = json.load(f)
dif = False
# for key,i in enumerate(x): 
#     for key2,j in enumerate(i['annotations']):
#         if j['word'] == 'None':
#             continue
#         else:
#             dif = True
#             start_index = i['text'].index(j['word'])
#             end_index = start_index + len(j['word'])
#             if j['start']!=start_index or j['end']!=end_index:
#                 dif = True
#                 # print(i)
#                 # print(x[key]['annotations'])
#                 x[key]['annotations'][key2]['start'] = start_index
#                 x[key]['annotations'][key2]['end']=end_index
# # Directly from dictionary
# if dif is True:    
#     with open('training_data.json', 'w') as outfile:
#         json.dump(x, outfile)

tr_d=[]
TRAIN_DATA=[]
for i in x:
    training_data = {'entities':[]}
    for j in i['annotations']:

        training_data['entities'].append((j['start'], j['end'], j['label']))
    # print(training_data)
    TRAIN_DATA.append((i['text'],training_data))


In [34]:
# adding labels to ner
model_run = False
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])
example = []
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    model_run = True
    optimizer = nlp.begin_training()
    
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update(
                [example], 
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        # print(losses)

100%|██████████| 108/108 [00:01<00:00, 59.02it/s]


In [35]:
if output_dir is not None and model_run is True:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    model_run = False
    print("Saved model to", output_dir)
pickle.dump(nlp, open( "models/model_custom_nlp.pkl", "wb" ))

Saved model to ner


In [36]:
doc=nlp("in three days i want to take a shared car from via Paisiello 26 to Viale Elvezia, 18")
for ent in doc.ents:
    print(ent.label_+ '  ------>   ' + ent.text)

DATE  ------>   in three days
TRANSPORT  ------>   shared car
LOC-FROM  ------>   via Paisiello 26
LOC-TO  ------>   Viale Elvezia, 18
